In [17]:
import geopandas as gpd
from owslib.wfs import WebFeatureService
import os

In [8]:
wfs_url="http://sit.comune.bolzano.it/geoserver/wfs?service=WFS"

In [9]:
wfs = WebFeatureService(url=wfs_url, version='2.0.0')

In [10]:
layers = list(wfs.contents)

In [11]:
incidenti = []
for layer in layers:
    if layer.find('ciden') > -1:
        incidenti.append(layer)

In [15]:
for incidente in incidenti:
    response = wfs.getfeature(typename=incidente)
    data = gpd.read_file(response)
    data.crs="epsg:25832"
    data = data.to_crs(epsg=4326)
    data['DTINCID']=data['DTINCID'].astype(str)
    data.DTINCID =data.DTINCID.apply(lambda x: x.replace(' 00:00:00+00:00',""))
    data['lon'] = data.geometry.x
    data['lat'] = data.geometry.y
    name = incidente.replace("Cartografia:view_","")
    data.to_file("data" + os.sep + name + ".geojson",driver="GeoJSON")
    del data['geometry']
    data.to_csv("data" + os.sep + name + ".csv",index=False)